# Supervised Learning — Single Source of Truth (Expanded Edition)

**This notebook contains:**

- Deep theoretical explanations with LaTeX-rendered formulas.
- Mathematical derivations and proofs for core algorithms.
- Visualizations (Matplotlib) for key formulas: Gaussian PDFs, Entropy, Sigmoid, SVM margin, Linear fit.
- Practical, runnable scikit-learn examples using real and synthetic datasets.
- Step-by-step commented code and a final comparison summary.

**Algorithms covered:** Naive Bayes, Performance Evaluation, Naive Bayes Optimizations, KNN, Decision Trees, Linear Regression, Logistic Regression, Support Vector Machines.

---

_Generated programmatically. Run cells in order to reproduce figures and analyses._

## Table of Contents

1. [Naive Bayes](#Naive-Bayes)
2. [Performance Evaluation](#Performance-Evaluation)
3. [Naive Bayes Optimizations](#Naive-Bayes-Optimizations)
4. [K-Nearest Neighbors (KNN)](#K-Nearest-Neighbors)
5. [Decision Trees](#Decision-Trees)
6. [Linear Regression](#Linear-Regression)
7. [Logistic Regression](#Logistic-Regression)
8. [Support Vector Machine (SVM)](#Support-Vector-Machine)
9. [Summary Comparison Table](#Summary-Comparison)
10. [References](#References)


In [ ]:
# Common imports used across examples
import numpy as np
import matplotlib.pyplot as plt
from math import log, sqrt, exp
import seaborn as sns

# Ensure plots show inline when run in a Jupyter environment
%matplotlib inline

print('Libraries imported. Run individual sections to reproduce figures and experiments.')

## Naive Bayes

### Definition & Intuition

Naive Bayes is a family of simple probabilistic classifiers based on **Bayes' theorem** with the strong (naive) assumption that features are conditionally independent given the class label. Despite this simplification, Naive Bayes often performs well in practice, especially on high-dimensional tasks such as text classification.

### Bayes' Theorem

$$
P(C_k \mid X) = \frac{P(X \mid C_k) P(C_k)}{P(X)}
$$

where \(X=(x_1,\dots,x_n)\) are features and \(C_k\) is the k-th class.

Under the naive independence assumption:

$$
P(X \mid C_k) = \prod_{i=1}^n P(x_i \mid C_k)
$$

so posterior becomes (up to normalization):

$$
P(C_k \mid X) \propto P(C_k) \prod_{i=1}^n P(x_i \mid C_k)
$$

### Variants and Likelihood Forms

- **Gaussian Naive Bayes (continuous features):**

$$
P(x_i \mid C_k) = \frac{1}{\sqrt{2\pi \sigma_{k,i}^2}} \exp\left(-\frac{(x_i-\mu_{k,i})^2}{2\sigma_{k,i}^2}\right)
$$

- **Multinomial Naive Bayes (counts):** likelihood proportional to counts of tokens given class.

- **Bernoulli Naive Bayes (binary features):** models presence/absence of features.

### Working Steps (Gaussian NB example)

1. Estimate class priors \(\hat{P}(C_k) = \frac{N_k}{N}\).
2. For each class and each feature, estimate mean \(\mu_{k,i}\) and variance \(\sigma_{k,i}^2\) from training data.
3. For a test sample compute class-conditional likelihoods using Gaussian density and multiply by priors.
4. Predict class with highest posterior.

### Mathematical note on log-probabilities

Because product of many probabilities can underflow, we usually compute log-probabilities:

$$
\log P(C_k \mid X) = \log P(C_k) + \sum_{i=1}^n \log P(x_i \mid C_k) + \text{constant}
$$

### Proof sketch (why multiply likelihoods?)

From conditional independence: \(P(x_1,x_2|C)=P(x_1|C)P(x_2|C)\). Repeated application yields product across features. This is simply the chain rule simplified by independence assumptions.

### Visualization objectives

- Plot Gaussian class-conditional densities for two classes along a single feature axis to show separation.
- Show how class posterior changes with x.

### Example & Code


In [ ]:
# Naive Bayes visualization and Iris example
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report

# 1D Gaussian PDF visualization for two classes
def gaussian_pdf(x, mu, sigma):
    return (1.0 / (np.sqrt(2 * np.pi) * sigma)) * np.exp(-0.5 * ((x - mu) / sigma) ** 2)

x = np.linspace(-5, 10, 400)
mu0, sigma0 = 0.0, 1.0
mu1, sigma1 = 3.0, 1.2
y0 = gaussian_pdf(x, mu0, sigma0)
y1 = gaussian_pdf(x, mu1, sigma1)

plt.figure(figsize=(8,4))
plt.plot(x, y0)
plt.plot(x, y1)
plt.title('Gaussian class-conditional densities (example)')
plt.xlabel('feature value (x)')
plt.ylabel('density')
plt.legend(['class 0', 'class 1'])
plt.show()

# Posterior example with equal priors
prior0 = 0.5
prior1 = 0.5
post0 = prior0 * y0
post1 = prior1 * y1
norm = post0 + post1
posterior0 = post0 / norm
posterior1 = post1 / norm

plt.figure(figsize=(8,3))
plt.plot(x, posterior0)
plt.plot(x, posterior1)
plt.title('Posterior probabilities for two classes (equal priors)')
plt.xlabel('feature value (x)')
plt.ylabel('P(class | x)')
plt.legend(['P(class0|x)', 'P(class1|x)'])
plt.show()

# Practical example: GaussianNB on Iris (first two features for visualization)
iris = load_iris()
X = iris.data[:, :2]  # take two features for easy plotting
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
gnb = GaussianNB()
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print('GaussianNB on Iris (2 features):')
print(classification_report(y_test, y_pred))

# Decision boundary visualization helper
def plot_decision_boundary(clf, X, y, title='Decision boundary'):
    # create a mesh to plot in
    x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
    y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200))
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(6,5))
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X[:,0], X[:,1], c=y)
    plt.title(title)
    plt.show()

plot_decision_boundary(gnb, X_test, y_test, title='GaussianNB Decision Boundary (Iris, 2 features)')


## Performance Evaluation (Expanded)

### Why it matters

Performance metrics quantify how well a model will perform on unseen data and guide choices such as which model to deploy, which hyperparameters to prefer, and how to handle class imbalance.

### Confusion matrix and derived metrics

Confusion matrix for binary classification:

\[
\begin{pmatrix}
TP & FP \\
FN & TN
\end{pmatrix}
\]

Formulas:

$$Accuracy=\frac{TP + TN}{TP + TN + FP + FN}$$

$$Precision=\frac{TP}{TP+FP},\quad Recall=\frac{TP}{TP+FN}$$

$$F1=2\cdot\frac{Precision\cdot Recall}{Precision+Recall}$$

### ROC and AUC

- Receiver Operating Characteristic (ROC) curve plots True Positive Rate vs False Positive Rate as the threshold varies.
- AUC is the area under ROC, a threshold-independent measure of separability.

### Cross-validation

- k-fold cross-validation: split data into k parts, train on k-1 parts, validate on the remaining part, repeat k times.
- Stratified k-fold preserves class ratios.

### Visualization objectives

- Plot ROC curve for a classifier on a binary problem.
- Show how precision/recall change with threshold (Precision-Recall curve).


In [ ]:
# Performance evaluation visualizations
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, precision_recall_curve

# synthetic binary dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=3, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
clf = LogisticRegression(max_iter=300)
clf.fit(X_train, y_train)
y_score = clf.predict_proba(X_test)[:,1]

fpr, tpr, _ = roc_curve(y_test, y_score)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(6,4))
plt.plot(fpr, tpr)
plt.plot([0,1],[0,1],'--')
plt.title(f'ROC curve (AUC = {roc_auc:.3f})')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

precision, recall, _ = precision_recall_curve(y_test, y_score)
plt.figure(figsize=(6,4))
plt.plot(recall, precision)
plt.title('Precision-Recall curve')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()


## Naive Bayes Optimizations (Expanded)

### Smoothing

Laplace (additive) smoothing avoids zero probabilities by adding \(\alpha\) to counts:

$$P(w\mid C)=\frac{count(w,C)+\alpha}{\sum_{w'}count(w',C)+\alpha |V|}$$

### Feature selection & dimensionality reduction

- Techniques such as chi-squared test, mutual information, and TF-IDF weighting improve Naive Bayes on text tasks.

### Calibration and Ensembles

- Calibration (e.g., isotonic regression, Platt scaling) can make predicted probabilities more accurate.
- Ensembles combining Naive Bayes with other weak learners can boost performance.

### Visualization goals

- Show effect of smoothing on probability estimates for rare tokens.


In [ ]:
# Demonstrate Laplace smoothing effect on token probability estimates
from collections import Counter

docs_spam = ['buy cheap meds', 'limited offer buy now', 'cheap meds available']
docs_ham = ['project meeting schedule', 'discussion about project', 'let us meet']
vocab = set(' '.join(docs_spam + docs_ham).split())
V = len(vocab)

def estimate_prob(token, docs, alpha):
    counts = Counter(' '.join(docs).split())
    total = sum(counts.values())
    return (counts[token] + alpha) / (total + alpha * V)

tokens = list(vocab)
for alpha in [0.0, 0.5, 1.0]:
    probs = [estimate_prob(t, docs_spam, alpha) for t in tokens]
    print(f'alpha={alpha}, sample probs (first 5):', probs[:5])


## K-Nearest Neighbors (KNN) — Expanded

### Working principle

KNN stores all training samples and, for a query point, finds the k nearest neighbors by a distance metric and predicts by majority vote (classification) or averaging (regression).

### Distance metrics

- Euclidean: \(d(x,y)=\sqrt{\sum (x_i-y_i)^2}\)
- Manhattan: \(d(x,y)=\sum |x_i-y_i|\)
- Cosine similarity: measures angle between vectors, useful for high-dimensional sparse data.

### Complexity

- Training: O(1) (stores data)
- Prediction: O(n * d) per query (can be reduced with KD-trees or Ball trees for low to moderate dimensions).

### Visualization goals

- Show how decision boundary changes as k varies on a 2D toy dataset.


In [ ]:
# KNN visualization: decision boundary for different k values
from sklearn.datasets import make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

X, y = make_classification(n_samples=200, n_features=2, n_redundant=0, n_clusters_per_class=1, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

def plot_knn_k(k):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train, y_train)
    # mesh
    x_min, x_max = X[:,0].min()-1, X[:,0].max()+1
    y_min, y_max = X[:,1].min()-1, X[:,1].max()+1
    xx, yy = np.meshgrid(np.linspace(x_min,x_max,200), np.linspace(y_min,y_max,200))
    Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(5,4))
    plt.contourf(xx, yy, Z, alpha=0.3)
    plt.scatter(X[:,0], X[:,1], c=y)
    plt.title(f'KNN decision boundary (k={k})')
    plt.show()

for k in [1,3,9]:
    plot_knn_k(k)


## Decision Trees — Expanded

### Split criteria

- **Entropy:**

$$Entropy(S) = -\sum_{i=1}^c p_i \log_2 p_i$$

- **Information Gain:** reduction in entropy after a split.

- **Gini impurity:**

$$Gini(S) = 1 - \sum_{i=1}^c p_i^2$$

### Proof sketch: why choose the split with highest information gain?

Splitting that reduces uncertainty (entropy) the most yields children that are purer and thus easier to classify. Information gain is simply parent entropy minus weighted child entropy.

### Visualization goals

- Plot entropy vs class probability p to show where entropy is highest (p=0.5).
- Train a small tree and visualize splits.


In [ ]:
# Decision tree entropy plot and small example
ps = np.linspace(0,1,500)
entropy = - (ps * np.log2(ps + 1e-12) + (1-ps) * np.log2(1-ps + 1e-12))
plt.figure(figsize=(6,3))
plt.plot(ps, entropy)
plt.title('Binary entropy as a function of p')
plt.xlabel('p')
plt.ylabel('Entropy')
plt.show()

# Small decision tree on synthetic data
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=200, noise=0.25, random_state=42)
dt = DecisionTreeClassifier(max_depth=3, random_state=42)
dt.fit(X, y)
plt.figure(figsize=(6,4))
plot_tree(dt, filled=True)
plt.title('Decision Tree (make_moons)')
plt.show()


## Linear Regression — Expanded

### Model and OLS derivation (normal equation)

Model: $$y = X\beta + \epsilon$$
Ordinary Least Squares minimizes RSS:

$$J(\beta) = (y - X\beta)^T(y - X\beta)$$
Set derivative w.r.t. \(\beta\) to zero to get normal equation:

$$-2X^T(y - X\beta) = 0 \Rightarrow X^TX\beta = X^T y$$
Provided \(X^TX\) invertible:

$$\hat{\beta} = (X^T X)^{-1} X^T y$$

### Visualization goals

- Fit a simple linear regression to noisy data and plot the best-fit line.


In [ ]:
# Linear regression fit and visualization
np.random.seed(42)
X = np.linspace(0, 10, 50)
y = 2.5 * X + 1.0 + np.random.normal(scale=3.0, size=X.shape)
X_mat = X.reshape(-1,1)

from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(X_mat, y)
y_pred = lr.predict(X_mat)
plt.figure(figsize=(6,4))
plt.scatter(X, y)
plt.plot(X, y_pred)
plt.title('Linear Regression fit')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

print('Estimated coefficient:', lr.coef_, 'intercept:', lr.intercept_)


## Logistic Regression — Expanded

### Model and likelihood

Logistic regression models probability via the sigmoid function:

$$\sigma(z) = \frac{1}{1 + e^{-z}}$$

Given dataset, the likelihood of parameters is:

$$L(\beta) = \prod_{i=1}^m \sigma(x^{(i)}\beta)^{y^{(i)}} (1-\sigma(x^{(i)}\beta))^{1-y^{(i)}}$$
Negative log-likelihood (loss):

$$J(\beta) = -\sum_{i=1}^m \left[y^{(i)}\log \sigma(x^{(i)}\beta) + (1-y^{(i)})\log(1-\sigma(x^{(i)}\beta))\right]$$

### Visualization goals

- Plot the sigmoid function and show how linear input maps to probability.


In [ ]:
# Sigmoid visualization and logistic regression example
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

z = np.linspace(-10, 10, 400)
plt.figure(figsize=(6,3))
plt.plot(z, sigmoid(z))
plt.title('Sigmoid function')
plt.xlabel('z')
plt.ylabel('sigma(z)')
plt.grid(True)
plt.show()

# Logistic regression on synthetic separable data
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
X, y = make_classification(n_samples=200, n_features=2, n_redundant=0, n_informative=2, random_state=42)
clf = LogisticRegression()
clf.fit(X, y)
print('Logistic Regression score:', clf.score(X, y))

# Plot decision boundary
xx, yy = np.meshgrid(np.linspace(X[:,0].min()-1, X[:,0].max()+1, 200), np.linspace(X[:,1].min()-1, X[:,1].max()+1, 200))
Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)
plt.figure(figsize=(6,4))
plt.contourf(xx, yy, Z, alpha=0.3)
plt.scatter(X[:,0], X[:,1], c=y)
plt.title('Logistic Regression decision boundary')
plt.show()


## Support Vector Machine (SVM) — Expanded

### Hard-margin objective (linearly separable)

Minimize:

$$\frac{1}{2} ||w||^2$$

Subject to:

$$y_i(w\cdot x_i + b) \ge 1, \quad \forall i$$

Margin between classes equals \(2/||w||\).

### Soft-margin (non-separable)

Introduce slack variables \(\xi_i\) and penalty parameter \(C\):

$$\min \frac{1}{2}||w||^2 + C \sum_i \xi_i$$
subject to \(y_i(w\cdot x_i + b) \ge 1 - \xi_i, \xi_i \ge 0\).

### Kernel trick

Replace dot product with kernel function \(K(x,x')\) to operate in implicit feature space.

### Visualization goals

- Plot SVM decision boundary, support vectors, and margin lines on a 2D dataset.


In [ ]:
# SVM margin visualization
from sklearn.svm import SVC
from sklearn.datasets import make_blobs

X, y = make_blobs(n_samples=100, centers=2, random_state=42)
clf = SVC(kernel='linear', C=1.0)
clf.fit(X, y)
plt.figure(figsize=(6,4))
xx = np.linspace(X[:,0].min()-1, X[:,0].max()+1, 200)
yy = np.linspace(X[:,1].min()-1, X[:,1].max()+1, 200)
XX, YY = np.meshgrid(xx, yy)
Z = clf.predict(np.c_[XX.ravel(), YY.ravel()])
Z = Z.reshape(XX.shape)
plt.contourf(XX, YY, Z, alpha=0.3)
plt.scatter(X[:,0], X[:,1], c=y)
# plot support vectors
sv = clf.support_vectors_
plt.scatter(sv[:,0], sv[:,1], s=100, facecolors='none', edgecolors='k')
plt.title('Linear SVM with support vectors (circled)')
plt.show()

print('Number of support vectors for each class:', clf.n_support_)


## Summary Comparison

| Algorithm | Type | Key Idea | Strength | Limitation | Common Use Case |
|---|---|---|---|---|---|
| Naive Bayes | Probabilistic | Bayes rule with feature independence | Fast, works well on text | Independence assumption | Spam detection |
| KNN | Instance-based | Majority vote among neighbors | Simple, non-parametric | Slow at prediction | Small-scale classification |
| Decision Tree | Tree-based | Recursive partitioning | Interpretable | Overfits easily | Classification with mixed features |
| Linear Regression | Parametric (regression) | Fit linear model via OLS | Interpretable | Poor for non-linear data | Trend estimation |
| Logistic Regression | Parametric (classification) | Linear model on log-odds via sigmoid | Probabilistic outputs | Linear decision boundary | Binary classification |
| SVM | Margin-based | Maximize margin using support vectors | Effective in high-dimensions | Slow on large datasets | Text classification, bioinformatics |


## References

- Bishop, C. M. *Pattern Recognition and Machine Learning*.
- Hastie, Tibshirani, Friedman. *The Elements of Statistical Learning*.
- scikit-learn documentation: https://scikit-learn.org
- Goodfellow, Bengio, Courville. *Deep Learning* (for broader context).
